# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 31 2022 11:40AM,245428,10,0085954816,ISDIN Corporation,Released
1,Aug 31 2022 11:40AM,245428,10,0085954817,ISDIN Corporation,Released
2,Aug 31 2022 11:40AM,245428,10,0085954826,ISDIN Corporation,Released
3,Aug 31 2022 11:40AM,245428,10,0085954827,ISDIN Corporation,Released
4,Aug 31 2022 11:40AM,245428,10,0085954834,ISDIN Corporation,Released
5,Aug 31 2022 11:40AM,245428,10,0085954835,ISDIN Corporation,Released
6,Aug 31 2022 11:40AM,245428,10,0085954836,ISDIN Corporation,Released
7,Aug 31 2022 11:40AM,245428,10,0085954848,ISDIN Corporation,Released
8,Aug 31 2022 11:40AM,245428,10,0085954849,ISDIN Corporation,Released
9,Aug 31 2022 11:40AM,245428,10,0085954884,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,245428,Released,14
27,245429,Released,1
28,245430,Released,1
29,245431,Released,1
30,245432,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
245428,NaN,14.0
245429,NaN,1.0
245430,NaN,1.0
245431,NaN,1.0
245432,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244946,20.0,46.0
245276,1.0,0.0
245281,0.0,1.0
245282,0.0,1.0
245284,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244946,20,46
245276,1,0
245281,0,1
245282,0,1
245284,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244946,20,46
1,245276,1,0
2,245281,0,1
3,245282,0,1
4,245284,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244946,20,46
1,245276,1,
2,245281,,1
3,245282,,1
4,245284,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation
14,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc."
15,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc."
16,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc."
17,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc."
18,Aug 31 2022 11:36AM,245427,19,"AdvaGen Pharma, Ltd"
46,Aug 31 2022 11:31AM,245426,22,"NBTY Global, Inc."
47,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc."
48,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc."
54,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,,14
1,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",,1
2,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",,1
3,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",,1
4,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",,1
5,Aug 31 2022 11:36AM,245427,19,"AdvaGen Pharma, Ltd",8,20
6,Aug 31 2022 11:31AM,245426,22,"NBTY Global, Inc.",,1
7,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",,1
8,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",,6
9,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",,22


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,14,
1,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",1,
2,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1,
3,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",1,
4,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",1,
5,Aug 31 2022 11:36AM,245427,19,"AdvaGen Pharma, Ltd",20,8
6,Aug 31 2022 11:31AM,245426,22,"NBTY Global, Inc.",1,
7,Aug 31 2022 10:56AM,245415,21,"NBTY Global, Inc.",1,
8,Aug 31 2022 10:56AM,245425,10,"Methapharm, Inc.",6,
9,Aug 31 2022 10:56AM,245424,10,"Methapharm, Inc.",22,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,14,
1,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",1,
2,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1,
3,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",1,
4,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,14.0,NaN
1,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",1.0,NaN
2,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1.0,NaN
3,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",1.0,NaN
4,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 31 2022 11:40AM,245428,10,ISDIN Corporation,14.0,0.0
1,Aug 31 2022 11:39AM,245432,22,"NBTY Global, Inc.",1.0,0.0
2,Aug 31 2022 11:39AM,245431,22,"NBTY Global, Inc.",1.0,0.0
3,Aug 31 2022 11:39AM,245430,22,"NBTY Global, Inc.",1.0,0.0
4,Aug 31 2022 11:38AM,245429,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1963332,53.0,21.0
15,490365,75.0,21.0
19,245427,20.0,8.0
20,245403,18.0,4.0
21,1962654,6.0,2.0
22,1227148,5.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1963332,53.0,21.0
1,15,490365,75.0,21.0
2,19,245427,20.0,8.0
3,20,245403,18.0,4.0
4,21,1962654,6.0,2.0
5,22,1227148,5.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,53.0,21.0
1,15,75.0,21.0
2,19,20.0,8.0
3,20,18.0,4.0
4,21,6.0,2.0
5,22,5.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,53.0
1,15,Released,75.0
2,19,Released,20.0
3,20,Released,18.0
4,21,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Executing,21.0,21.0,8.0,4.0,2.0,0.0
Released,53.0,75.0,20.0,18.0,6.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Executing,21.0,21.0,8.0,4.0,2.0,0.0
1,Released,53.0,75.0,20.0,18.0,6.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Executing,21.0,21.0,8.0,4.0,2.0,0.0
1,Released,53.0,75.0,20.0,18.0,6.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()